# Upper Confidence Bound (UCB)
We will be using UCB to solve the Multi-Armed Bandit Problem.

The confidence bound bounds the current and expected return values.
- For each iteration (round), the UCB algorithm picks the machine with the greatest upper confidence bound

### How Upper Confidence Bound Changes
If the player wins a round, then the confidence bound and current return value of the machine shifts up.  
If the player loses a round, then the confidence bound and current return value of the machine shifts down.

As each round is played, the confidence bound of the machine played shrinks because the program is more confident that the current return value is close to the expected return value.

After enough iterations, the algorithm converges to the most optimal machine to play because that machine's upper confidence bound will always be the greatest out of the other confidence bounds.

# UCB Algorithm Visualization